[Dive into CLVM](https://chialisp.com/docs/clvm/dive_into_clvm)

## [Lazy Evaluation in CLVM](https://chialisp.com/docs/clvm/dive_into_clvm#lazy-evaluation-in-clvm)

### eager evaluation
```sh
❯ brun '(i (q . 1) (q . "true") (x))'
FAIL: clvm raise ()

❯ run '(mod () (if (1) "true" (x)))'
(x)
```

### lazy evaluation

`(a (i (A) (q . B) (q . C)) 1)`

```sh
❯ brun '(a (i (q . 1) (q . (q . "true")) (q . (x))) 1)'
"true"

chia-concepts/notebooks/clvm on  singleton-tic-tac-toe [?] 
❯ brun '(a (i (q . 0) (q . (q . "true")) (q . (x))) 1)'  
FAIL: clvm raise ()

❯ run '(mod (value) (if (= value 1) "true" (x)))'          
(a (i (= 2 (q . 1)) (q 1 . "true") (q 8)) 1)

❯ brun '(a (i (= 2 (q . 1)) (q 1 . "true") (q 8)) 1)' '(1)'  
"true"

❯ brun '(a (i (= 2 (q . 1)) (q 1 . "true") (q 8)) 1)' '(0)' 
FAIL: clvm raise ()

❯ brun '(a (i (= 2 (q . 1)) (q 1 . "true") (q 8)) 1)' '(12)'
FAIL: clvm raise ()

❯ brun '(a (i (= 2 (q . 1)) (q . (q . "true")) (q . (x))) 1)' '(1)'
"true"

❯ brun '(a (i (= 2 (q . 1)) (q . (q . "true")) (q . (x))) 1)' '(0)'
FAIL: clvm raise ()
```

## Evaluate

### example 1
```c
(a 
     (q . (+ 2 (q . 5))) 
     (q . (70 80 90))
) 
```

```sh
❯ brun '(a (q . (+ 2 (q . 5))) (q . (70 80 90)))' '(20 30 40)'
75
```
#### step 1

#### Accessing Environmental Variables Through Integers ([1](https://chialisp.com/docs/clvm/basics#accessing-environmental-variables-through-integers), [2](https://developers.chia.net/t/accessing-environmental-variables-through-integers-hard-to-understand-what-it-means/210/2))

```c
1 -> (70 80 90)
2 -> (f 1) -> 70
```

#### step 2
```c
(+ 70 5) -> 75
```

### example 2

```sh
❯ brun '(a (q . (+ 2 (q . 5))) (c 2 (q . (70 80 90))))' '(20 30 40)'
25
```

#### step 1

> 20 is [divmod](https://github.com/Chia-Network/clvm_rs/blob/main/src/chia_dialect.rs#L71)

```sh
❯ brun '(c 2 (q . (70 80 90)))' '(20 30 40)'
(divmod 70 80 90)
```

#### step 2
```c
(+ 20 5) -> 25
```

## Programs as Parameters
### example 1

```sh
❯ brun '(a 2 (q . (12)))' '((* 2 (q . 2)))'
24
```

```sh
(a 2 (q . (12)))
-> (a (q . (* 2 (q . 2))) (q . (12)))
-> (* 12 2)
-> 24
```

```sh
❯ brun '(a (q . (* 2 (q . 2))) (q . (12)))'       
24
```

### example 2
```sh
❯ brun '(a 2 1)' '((* 5 (q . 2)) 10)'
20
```

```sh
1 -> ((* 5 (q . 2)) 10)
2 -> (f 1) -> (* 5 (q . 2))
5 -> (f (r 1)) -> (f (10)) -> 10

(a 2 1)
-> (a (q . (* 5 (q . 2))) (q . ((* 5 (q . 2)) 10)))
-> (* 10 2)
-> 20
```
